In [8]:
import yfinance as yf
import pandas as pd
import numpy as np

In [9]:
tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN", "NVDA", "META", "NFLX", "JPM", "XOM", "KO"]

data=yf.download(tickers, period="1y", interval="1d", group_by='ticker', auto_adjust=False)

ticker_dfs={}
for ticker in tickers:
    df=data[ticker].copy()
    df.dropna(inplace=True)
    ticker_dfs[ticker]=df


[*********************100%***********************]  11 of 11 completed


In [10]:
def compute_ratios(df):
    ratios = {}

    # Daily % Change
    df["pct_change"] = df["Adj Close"].pct_change()

    # Rolling % Changes
    for window, name in [(1, "1D"), (5, "5D"), (21, "1M"), (63, "3M"), (126, "6M"), (252, "1Y")]:
        ratios[f"pct_change_{name}"] = (df["Adj Close"].pct_change(periods=window).iloc[-1])

    # Moving Averages
    for window in [20, 50, 200]:
        ratios[f"SMA_{window}"] = df["Adj Close"].rolling(window).mean().iloc[-1]

    for window in [12, 26]:
        ratios[f"EMA_{window}"] = df["Adj Close"].ewm(span=window, adjust=False).mean().iloc[-1]

    # MACD
    ema12 = df["Adj Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Adj Close"].ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    ratios["MACD"] = macd.iloc[-1]
    ratios["MACD_signal"] = signal.iloc[-1]

    # RSI
    delta = df["Adj Close"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    ratios["RSI_14"] = 100 - (100 / (1 + rs.iloc[-1]))

    # Bollinger Bands
    sma20 = df["Adj Close"].rolling(20).mean()
    std20 = df["Adj Close"].rolling(20).std()
    ratios["Bollinger_upper"] = sma20.iloc[-1] + (2 * std20.iloc[-1])
    ratios["Bollinger_lower"] = sma20.iloc[-1] - (2 * std20.iloc[-1])

    # ATR
    high_low = df["High"] - df["Low"]
    high_close = (df["High"] - df["Adj Close"].shift()).abs()
    low_close = (df["Low"] - df["Adj Close"].shift()).abs()
    tr = high_low.to_frame(name="hl")
    tr["hc"] = high_close
    tr["lc"] = low_close
    tr = tr.max(axis=1)
    ratios["ATR_14"] = tr.rolling(14).mean().iloc[-1]

    # Historical Volatility (30D)
    log_returns = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(lambda x: np.log(x))
    ratios["HistVol_30D"] = log_returns.rolling(30).std().iloc[-1] * (252 ** 0.5)

    # Volume Averages
    ratios["Vol_Avg_20D"] = df["Volume"].rolling(20).mean().iloc[-1]
    ratios["Vol_Last"] = df["Volume"].iloc[-1]

    # On-Balance Volume (OBV)
    obv = (np.sign(df["Adj Close"].diff()) * df["Volume"]).fillna(0).cumsum()
    ratios["OBV"] = obv.iloc[-1]

    return ratios

In [11]:
ratios_dict = {}

for ticker, df in ticker_dfs.items():
    ratios_df = compute_ratios(df)  
    ratios_dict[ticker] = ratios_df

In [12]:
import pprint
pprint.pprint(ratios_dict)

{'AAPL': {'ATR_14': np.float64(4.464287894112723),
          'Bollinger_lower': np.float64(222.73502571412564),
          'Bollinger_upper': np.float64(240.5509728210306),
          'EMA_12': np.float64(232.31586132440032),
          'EMA_26': np.float64(228.55286304428722),
          'HistVol_30D': np.float64(0.2938535811730464),
          'MACD': np.float64(3.762998280113095),
          'MACD_signal': np.float64(4.787058217862571),
          'OBV': np.float64(225162200.0),
          'RSI_14': np.float64(56.751784945407344),
          'SMA_20': np.float64(231.64299926757812),
          'SMA_200': np.float64(221.02951309204101),
          'SMA_50': np.float64(220.56370330810546),
          'Vol_Avg_20D': np.float64(47816790.0),
          'Vol_Last': np.int64(50150900),
          'pct_change_1D': np.float64(0.014286368839370622),
          'pct_change_1M': np.float64(0.0016547132284436827),
          'pct_change_1Y': np.float64(nan),
          'pct_change_3M': np.float64(0.1585223045141

In [13]:
import numpy as np

def interpret_rsi(rsi):
    if rsi > 70:
        return "overbought"
    elif rsi < 30:
        return "oversold"
    else:
        return "neutral"

def interpret_macd(macd, signal):
    if macd > signal:
        return "bullish"
    elif macd < signal:
        return "bearish"
    else:
        return "neutral"

def interpret_volume(last, avg):
    if last > avg:
        return "higher than average"
    elif last < avg:
        return "lower than average"
    else:
        return "about average"

def interpret_obv(obv_change):
    if obv_change > 0:
        return "buying pressure"
    elif obv_change < 0:
        return "selling pressure"
    else:
        return "neutral"

def create_summary(ticker, data_dict):
    # Trend & Momentum
    trend_text = f"""[Trend & Momentum]
Price Changes:
- 1 Day: {data_dict['pct_change_1D']:.2%}, 5 Days: {data_dict['pct_change_5D']:.2%}, 1 Month: {data_dict['pct_change_1M']:.2%}, 3 Months: {data_dict['pct_change_3M']:.2%}, 6 Months: {data_dict['pct_change_6M']:.2%}, 1 Year: {data_dict.get('pct_change_1Y','N/A')}

SMA/EMA:
- SMA20: {data_dict['SMA_20']:.2f}, SMA50: {data_dict['SMA_50']:.2f}, SMA200: {data_dict['SMA_200']:.2f}
- EMA12: {data_dict['EMA_12']:.2f}, EMA26: {data_dict['EMA_26']:.2f}

Momentum Indicators:
- RSI(14): {data_dict['RSI_14']:.1f} ({interpret_rsi(data_dict['RSI_14'])})
- MACD: {data_dict['MACD']:.2f}, Signal Line: {data_dict['MACD_signal']:.2f} ({interpret_macd(data_dict['MACD'], data_dict['MACD_signal'])})
"""

    # Volatility & Risk
    volatility_text = f"""[Volatility & Risk]
Bollinger Bands (20-day):
- Lower: {data_dict['Bollinger_lower']:.2f}, Upper: {data_dict['Bollinger_upper']:.2f}
ATR(14): {data_dict['ATR_14']:.2f}
Historical Volatility (30D): {data_dict['HistVol_30D']:.2%}
Beta vs S&P500: {data_dict.get('Beta','N/A')}
"""

    # Volume Signals
    volume_text = f"""[Volume Signals]
Volume:
- 20-day Average: {data_dict['Vol_Avg_20D']:.0f}, Last Day: {data_dict['Vol_Last']:.0f} ({interpret_volume(data_dict['Vol_Last'], data_dict['Vol_Avg_20D'])})
OBV: {data_dict['OBV']:.0f} ({interpret_obv(data_dict['OBV'])})
"""

    return {
        "ticker": ticker,
        "trend": trend_text,
        "volatility": volatility_text,
        "volume": volume_text
    }


for ticker in tickers:
    summary = create_summary(ticker, ratios_dict[ticker])
    print(f"--- Summary for {ticker} ---")
    print(summary['trend'])
    print(summary['volatility'])
    print(summary['volume'])


--- Summary for AAPL ---
[Trend & Momentum]
Price Changes:
- 1 Day: 1.43%, 5 Days: -4.07%, 1 Month: 0.17%, 3 Months: 15.85%, 6 Months: 6.27%, 1 Year: nan

SMA/EMA:
- SMA20: 231.64, SMA50: 220.56, SMA200: 221.03
- EMA12: 232.32, EMA26: 228.55

Momentum Indicators:
- RSI(14): 56.8 (neutral)
- MACD: 3.76, Signal Line: 4.79 (bearish)

[Volatility & Risk]
Bollinger Bands (20-day):
- Lower: 222.74, Upper: 240.55
ATR(14): 4.46
Historical Volatility (30D): 29.39%
Beta vs S&P500: N/A

[Volume Signals]
Volume:
- 20-day Average: 47816790, Last Day: 50150900 (higher than average)
OBV: 225162200 (buying pressure)

--- Summary for MSFT ---
[Trend & Momentum]
Price Changes:
- 1 Day: 0.13%, 5 Days: -1.37%, 1 Month: -5.18%, 3 Months: 6.18%, 6 Months: 31.18%, 1 Year: nan

SMA/EMA:
- SMA20: 506.16, SMA50: 509.43, SMA200: 444.29
- EMA12: 503.00, EMA26: 506.04

Momentum Indicators:
- RSI(14): 46.2 (neutral)
- MACD: -3.04, Signal Line: -1.97 (bearish)

[Volatility & Risk]
Bollinger Bands (20-day):
- Lower: 

In [14]:
import pinecone
from pinecone import ServerlessSpec
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API=os.getenv("PINECONE_API")
pc=pinecone.Pinecone(api_key=PINECONE_API)
index_name="ticker-ratios"

if index_name not in pc.list_indexes().names():
            pc.create_index(
                name=index_name,
                dimension=384,
                metric='cosine',
                spec=ServerlessSpec(cloud='aws', region='us-east-1')
            )

index=pc.Index(index_name)
model=SentenceTransformer("all-MiniLM-L6-v2")

for ticker in tickers:
    summary = create_summary(ticker, ratios_dict[ticker])

    embedding_trend=model.encode(summary['trend'])
    embedding_volatility=model.encode(summary['volatility'])
    embedding_volume=model.encode(summary['volume'])

    upsert_items=[
  (f"{ticker}_trend", 
   embedding_trend, 
   {"text": summary["trend"], "ticker": ticker, "section": "trend"}),
  (f"{ticker}_volatility", 
   embedding_volatility, 
   {"text": summary["volatility"], "ticker": ticker, "section": "volatility"}),
  (f"{ticker}_volume", 
   embedding_volume, 
   {"text": summary["volume"], "ticker": ticker, "section": "volume"})]
    
    index.upsert(upsert_items)
    